# Transfer Learning

In [11]:
import torchvision
import torch.nn.functional as F
from torchvision.transforms import transforms
import torch
import matplotlib.pyplot as plt
from torch import nn
from icecream import ic

In [12]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

In [13]:
cifar_10 = torchvision.datasets.CIFAR10('.', download=True, transform=preprocess)
train_loader = torch.utils.data.DataLoader(cifar_10, batch_size=512, shuffle=True)

Using downloaded and verified file: ./cifar-10-python.tar.gz
Extracting ./cifar-10-python.tar.gz to .


In [15]:
# transfer step 1: load pretrained model
resnet = torchvision.models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False # frozen weights

In [16]:
feature_num = resnet.fc.in_features
resnet.fc = nn.Linear(feature_num, 10) # rewrite fc classifier

In [17]:
ic(resnet(cifar_10[0][0].unsqueeze(0)))

ic| resnet(cifar_10[0][0].unsqueeze(0)): tensor([[ 1.0818,  0.0746, -0.8494, -0.0205,  0.2240,  0.2281, -0.4149, -0.6761,
                                                  -0.6851,  0.8544]], grad_fn=<AddmmBackward0>)


tensor([[ 1.0818,  0.0746, -0.8494, -0.0205,  0.2240,  0.2281, -0.4149, -0.6761,
         -0.6851,  0.8544]], grad_fn=<AddmmBackward0>)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=1e-3, momentum=0.9)

In [19]:
epochs = 2
losses = []

In [20]:
for epoch in range(epochs):
    epoch_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        ic(epoch, i)
        output = resnet(images)
        loss = criterion(output, labels)
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

        if i > 0:
            print('Epoch: {} batch: {}, loss ==> {}'.format(epoch, i, epoch_loss/i))

    losses.append(epoch_loss / i)

ic| epoch: 0, i: 0
ic| epoch: 0, i: 1


Epoch: 0 batch: 1, loss ==> 5.0546557903289795


ic| epoch: 0, i: 2


In [ ]:
plt.plot(losses)